In [6]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict,Literal, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [7]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [8]:
llm = ChatOpenAI()

def chat_node_func(state: ChatState):

    #take user query form state
    messages = state['messages']
    #send to llm
    response = llm.invoke(messages)
    #store response to state
    return {'messages': [response]}

In [9]:

checkpointer = MemorySaver()
graph=StateGraph(ChatState)
graph.add_node('chat_node', chat_node_func)

graph.add_edge(START, 'chat_node')
graph.add_edge( 'chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

#graph.compile()



In [10]:
initial_state={
    'messages': [HumanMessage(content='What is the capital of India')]
}

chatbot.invoke(initial_state)['messages'][-1].content

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id

In [ ]:
thread_id = '1'
while True:
    user_message = input('Type here:')
    print('User: ', user_message)
    if user_message.strip().lower() in ['exit','quite','bye']:
        break

    config={'configurable':{'thread_id': thread_id}}
    response = chatbot.invoke({'messages': [HumanMessage(content=user_message)]}, config=config)

    print('AI:', response['messages'][-1].content)

User:  Hello My Name is Naresh
AI: Hello Naresh! How can I assist you today?
User:  I'm expert in AI
AI: That's great to hear, Naresh! It's always fantastic to connect with someone who is knowledgeable in the field of artificial intelligence. Is there anything specific you would like to discuss or share about AI?
User:  What is my Name?
AI: Your name is Naresh, as you mentioned in your first message. How can I assist you further, Naresh?
User:  What is my NAme?
AI: Your name is Naresh. Is there something specific you would like to know or discuss, Naresh?
User:  bye


In [ ]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='Hello My Name is Naresh', additional_kwargs={}, response_metadata={}, id='4936d208-be4d-4912-9c3e-d3936f264b49'), AIMessage(content='Hello Naresh! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CW0RbiuZVxRh9x2tROUXQn5YZAwSB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--154ac3da-e4c0-4a8a-952c-aa8a38bd1386-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'a